# Данный подход базируется на анализе исключительно текста. Без использования ключевых слов и критериев отклонения - принятия.
# Каркас - составление TF-IDF матрицы, затем ее сжатия до 100 признаков(с использованием SVD) и дальнейшим обучением RandomForest.

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
test=pd.read_pickle("test_df.pkl")
test.drop(columns=["html"],inplace=True)
test.head()

,id,text,keywords,accepted_function,rejected_function,accepted_product,rejected_product
0,071c2e15-5fd0-4e7d-b78d-26b7c34491e1,[We use cookies to tailor our service and ensu...,"[technologies, descriptive, mapping, learning,...","IT support, IT managed services, IT integratio...","manufacture, production, processing, distribut...",,
1,e534df73-b385-4a12-8368-38992d3af7d3,[About us\nAbout Powernet\nPartnerships\nCusto...,"[outsourcing, projects, IT, sensors, convergen...","software development, database development, sy...","wholesale, distribution, retail selling, manuf...",,
2,4e38c9ff-bffe-4251-b2b2-94777c7344fe,[Work\nApproach\nPeople\nValues\nBlog\nCareers...,"[Inventing, products, risk, development, servi...","general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,
3,a7f8870c-30f8-4851-8f8b-4c16942b397f,[Menu\nMenu\nSupport\nInfrastructure\nColocati...,"[low, chain, portfolio, services, Procurement,...","Investment research and information services, ...","Construction, production and supply, Advertisi...",,
4,c062f411-ae3e-486a-9cba-2267889adb2b,[Français (French)\nEnglish (English)\n日本語 (Ja...,"[recruitment, producing, Evaluation, Payroll, ...","general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,


In [3]:
train=pd.read_pickle("train_df.pkl")
train.drop(columns=["html"],inplace=True)
train.head()

,id,text,keywords,accepted_function,rejected_function,accepted_product,rejected_product,target
0,b811ea46-1bc5-4a18-a64c-7504c382da44,[Release\n2020\n2019\n2018\n2017\n2020\n2019\n...,"[tools, services, projects, incubates, investm...","accounting, bookeeping, auditing, finance, HR,...","manufacture production processing assembly, re...",,,2
1,b5e3f6d4-30bc-443c-a9d6-8ed41d5c33b3,[SAP Integrated Business Planning (IBP)\nSAP A...,"[budgeting, decision, cycles, closing, Simplif...","IT consultancy, IT support services, IT manage...","distribution, sale, trade, wholesale, retail s...",,,2
2,997ae1ab-93c4-45a9-9bd0-a49b2b952f1f,[facebook\ntwitter\nlinkedin\ninstagram\nWelco...,[],"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,,1
3,6890f2f1-bdc1-48cb-bf7f-00944eb37e77,[ANDERSON BUSINESS TRAVEL\nSpecialist Independ...,"[provide, service, financial, arranging, trave...","insurance brokerage, insurance agents, travel ...","software development, wholesale, retail sale, ...",,,2
4,74a95f04-e440-4af0-850a-eec5a44ba397,[Amari Ireland Ltd\nStandards\nPeople\nQuality...,"[independent, centre, forms, copper, bronze, s...","sales agent and sales representative, product ...","manufacture, production, processing and retail...","product, motor vehicles, buses and trucks, mot...",,1


In [4]:
for i in train.columns[:-1]:
    train[i]=train[i].apply(lambda x: np.nan if (x=="" or x==[]) else x   )
train.head()

,id,text,keywords,accepted_function,rejected_function,accepted_product,rejected_product,target
0,b811ea46-1bc5-4a18-a64c-7504c382da44,[Release\n2020\n2019\n2018\n2017\n2020\n2019\n...,"[tools, services, projects, incubates, investm...","accounting, bookeeping, auditing, finance, HR,...","manufacture production processing assembly, re...",NaN,NaN,2
1,b5e3f6d4-30bc-443c-a9d6-8ed41d5c33b3,[SAP Integrated Business Planning (IBP)\nSAP A...,"[budgeting, decision, cycles, closing, Simplif...","IT consultancy, IT support services, IT manage...","distribution, sale, trade, wholesale, retail s...",NaN,NaN,2
2,997ae1ab-93c4-45a9-9bd0-a49b2b952f1f,[facebook\ntwitter\nlinkedin\ninstagram\nWelco...,NaN,"general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",NaN,NaN,1
3,6890f2f1-bdc1-48cb-bf7f-00944eb37e77,[ANDERSON BUSINESS TRAVEL\nSpecialist Independ...,"[provide, service, financial, arranging, trave...","insurance brokerage, insurance agents, travel ...","software development, wholesale, retail sale, ...",NaN,NaN,2
4,74a95f04-e440-4af0-850a-eec5a44ba397,[Amari Ireland Ltd\nStandards\nPeople\nQuality...,"[independent, centre, forms, copper, bronze, s...","sales agent and sales representative, product ...","manufacture, production, processing and retail...","product, motor vehicles, buses and trucks, mot...",NaN,1


In [5]:
train['text']=train['text'].apply(lambda x: x[0].lower()+x[1].lower() if(len(x)==2) else x[0].lower())
test['text']=test['text'].apply(lambda x:x[0].lower()+x[1].lower() if(len(x)==2) else x[0].lower())

In [6]:
features_list=['accepted_function','accepted_product','rejected_function','rejected_product','keywords']
train.drop(columns=features_list, inplace=True)
test.drop(columns=features_list, inplace=True)
train.head()

,id,text,target
0,b811ea46-1bc5-4a18-a64c-7504c382da44,release\n2020\n2019\n2018\n2017\n2020\n2019\n2...,2
1,b5e3f6d4-30bc-443c-a9d6-8ed41d5c33b3,sap integrated business planning (ibp)\nsap ad...,2
2,997ae1ab-93c4-45a9-9bd0-a49b2b952f1f,facebook\ntwitter\nlinkedin\ninstagram\nwelcom...,1
3,6890f2f1-bdc1-48cb-bf7f-00944eb37e77,anderson business travel\nspecialist independe...,2
4,74a95f04-e440-4af0-850a-eec5a44ba397,amari ireland ltd\nstandards\npeople\nquality\...,1


In [7]:
test.head()

,id,text
0,071c2e15-5fd0-4e7d-b78d-26b7c34491e1,we use cookies to tailor our service and ensur...
1,e534df73-b385-4a12-8368-38992d3af7d3,about us\nabout powernet\npartnerships\ncustom...
2,4e38c9ff-bffe-4251-b2b2-94777c7344fe,work\napproach\npeople\nvalues\nblog\ncareers\...
3,a7f8870c-30f8-4851-8f8b-4c16942b397f,menu\nmenu\nsupport\ninfrastructure\ncolocatio...
4,c062f411-ae3e-486a-9cba-2267889adb2b,français (french)\nenglish (english)\n日本語 (jap...


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(analyzer="word",stop_words='english')
mat_train=tfidf.fit_transform(np.array(train['text']))
mat_test=tfidf.transform(np.array(test['text']))

In [9]:
mat_train

<15515x166688 sparse matrix of type '<class 'numpy.float64'>'
	with 4714969 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=100)
mat_train=svd.fit_transform(mat_train)
mat_test=svd.transform(mat_test)

In [11]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=1000,n_jobs=-1)
clf.fit(mat_train,np.array(train['target']))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [12]:
result=clf.predict(mat_test)

In [13]:
df=pd.DataFrame(data=np.hstack([np.array(test['id']).reshape(len(test),1),result.reshape(len(test),1)]),columns=['id','target'])
df.to_csv("result.csv",index=False)

# Что дает весьма приличный результат на Kaggle(несмотря на простоту идеи).
# public sample : 0.71206
# private sample: 0.69404
# (Победитель оценивается по результатам на private sample, которые становятся известными только после соревнований)